In [ ]:
import tensorflow as tf
import plotly.express as px
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications.efficientnet import preprocess_input
import matplotlib.pyplot as plt
from utils import *

In [ ]:
directory = 'images/final/'
width = 128
height = 128
channels = 4
image_size = (width, height)
input_shape = (width, height, channels)
batch_size = 32

In [ ]:
train_ds, class_names = load_data(directory, batch_size, image_size, GAN=True)

In [ ]:
train_ds = prepare(train_ds, shuffle=True, augment=True, GAN=True)

channels = 3
input_shape = (width, height, channels)

In [ ]:
# for image_batch, label_batch in train_ds.take(1):
#     print(image_batch[0][50][50])
#     im = image_batch * 127.5 + 127.5
#     print(im[0][50][50])
#     break

In [ ]:
plot_n_images(train_ds, 5, class_names, GAN=True)

In [ ]:
# define the discriminator model
def define_discriminator(in_shape=input_shape):
    model = tf.keras.Sequential()
    # normal
    model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', input_shape=in_shape))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    # downsample to 64x64
    model.add(tf.keras.layers.Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    # downsample to 32x32
    model.add(tf.keras.layers.Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    # downsample to 16x16
    model.add(tf.keras.layers.Conv2D(256, (3,3), strides=(2,2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    # downsample to 8x8
    model.add(tf.keras.layers.Conv2D(256, (3,3), strides=(2,2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    # classifier
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    # compile model
    opt = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model
    

In [ ]:
# define the standalone generator model
def define_generator(latent_dim):
    model = tf.keras.Sequential()
    # foundation for 8x8 feature maps
    n_nodes = 256 * 8 * 8
    model.add(tf.keras.layers.Dense(n_nodes, input_dim=latent_dim))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Reshape((8, 8, 256)))
    # upsample to 16x16
    model.add(tf.keras.layers.Conv2DTranspose(256, (4,4), strides=(2,2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    # upsample to 32x32
    model.add(tf.keras.layers.Conv2DTranspose(256, (4,4), strides=(2,2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    # upsample to 64x64
    model.add(tf.keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    # upsample to 128x128
    model.add(tf.keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    # output layer
    model.add(tf.keras.layers.Conv2D(3, (3,3), activation='tanh', padding='same'))
    return model

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = tf.keras.Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [8]:
# build and compile the discriminator
discriminator = define_discriminator()
discriminator.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 128)     3584      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 128, 128, 128)     0         
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 128)       147584    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 64, 64, 128)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 128)       147584    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 32, 32, 128)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 256)      

In [9]:
# build the generator
generator = define_generator(100)
generator.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 16384)             1654784   
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 16384)             0         
                                                                 
 reshape (Reshape)           (None, 8, 8, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 16, 16, 256)      1048832   
 nspose)                                                         
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 16, 16, 256)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 32, 32, 256)      1048832   
 ranspose)                                            

In [10]:
# build the gan
gan_model = define_gan(generator, discriminator)
gan_model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 128, 128, 3)       4542595   
                                                                 
 sequential_1 (Sequential)   (None, 1)                 1200385   
                                                                 
Total params: 5,742,980
Trainable params: 4,542,595
Non-trainable params: 1,200,385
_________________________________________________________________


In [17]:
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y = tf.zeros((n_samples, 1))
    return X, y

def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = tf.random.normal(shape=(n_samples, latent_dim))
    return x_input

def generate_real_samples(dataset, n_samples):
    X = []
    while len(X) < n_samples:
        dataset.shuffle(100, seed=123)
        for x, _ in dataset.take(5):
            for i in range(len(x)):
                X.append(x[i])
                if len(X) == n_samples:
                    break
            if len(X) == n_samples:
                break
    X = tf.stack(X)
    y = tf.ones((n_samples, 1))
    return X, y

def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save plot
    save_plot(x_fake, epoch)
    # save the generator model tile file
    filename = 'generator_model_e%03d.h5' % (epoch+1)
    g_model.save(filename)

def save_plot(examples, epoch, n=10):
    # plot images
    for i in range(n * n):
        # define subplot
        plt.subplot(n, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.imshow(examples[i, :, :, 0], cmap='gray_r')
    # save plot to file
    filename = 'generated_plot_e%03d.png' % (epoch+1)
    plt.savefig(filename)
    plt.close()

In [12]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=64):
    bat_per_epo = int(dataset.cardinality())
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j, batch in enumerate(dataset):
            # get randomly selected 'real' samples
            X_real, y_real = batch[0], batch[1]
            half_batch = int(X_real.shape[0])
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # create training set for the discriminator
            y_real = tf.reshape(y_real, (half_batch, 1))
            X, y = tf.concat([X_real, X_fake], axis=0), tf.concat([y_real, y_fake], axis=0)
            # update discriminator model weights
            d_loss, _ = d_model.train_on_batch(X, y)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = tf.ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
        # evaluate the model performance, sometimes
        if (i) % 100 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)
    # save the generator model
    g_model.save('generator_model_final.h5')


In [13]:
# size of the latent space
latent_dim = 100
# train model
# train(generator, discriminator, gan_model, train_ds, latent_dim, n_epochs=150)


test

In [46]:
model = tf.keras.models.load_model('generator_model_final.h5')

In [119]:
X = model.predict(generate_latent_points(100, 1))
plot_image(X[0], "image générée", {"real": float(discriminator.predict(X))})

1/1 [==============================] - 0s 19ms/step


In [88]:
X = model.predict(generate_latent_points(100, 100))
#show the 10 most confident predictions of the discriminator

    

4/4 [==============================] - 0s 67ms/step


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_7/conv2d_10/Conv2D' defined at (most recent call last):
    File "c:\Users\enzoh\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\enzoh\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\enzoh\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\enzoh\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1041, in launch_instance
      app.start()
    File "C:\Users\enzoh\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 724, in start
      self.io_loop.start()
    File "C:\Users\enzoh\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 215, in start
      se

# GAN only fire 



In [26]:
import pandas as pd

# Load the dataset with Pokemon attributes
pokemon_data = pd.read_csv("PokeDataset.csv")

import os
import numpy as np
from PIL import Image

def load_type(type,size=(128,128),directory="images/final"):
    # Get the names of all Pokemon that have "Fire" as their Type1 or Type2 attribute
    pokemon_names = set(pokemon_data[(pokemon_data['Type1'] == type) | (pokemon_data['Type2'] == type)]['Name'])
    # Filter train_ds to only include images of Pokemon that have "Fire" as either their Type1 or Type2 attribute

    # iterate trough folder images\final\ and put in a numpy array image in folder that are in water_pokemon_names


    # initialize an empty numpy array to store the images
    num_images = 0
    for subdir, dirs, files in os.walk(directory):
        for name in dirs:
            if name in pokemon_names:
                num_images += len(os.listdir(os.path.join(subdir, name)))
    images = np.empty((num_images, size[0], size[1], 4), dtype=np.float32)

    i = 0
    for subdir, dirs, files in os.walk(directory):
        # iterate through all subdirectories but only keep the ones that are in water_pokemon_names
        for name in dirs:
            if name in pokemon_names:
                # iterate through all images in the subdirectory
                for file in os.listdir(os.path.join(subdir, name)):
                    # open the image and resize it
                    img = Image.open(os.path.join(subdir, name, file))
                    img = img.resize(size)
                    # convert the image to RGBA
                    img = img.convert('RGBA')
                    # convert the image to a numpy array
                    img = np.array(img)
                    # add the image to the numpy array
                    images[i] = img
                    i += 1
    return images,num_images

In [27]:
images_size = (128, 128)
num_images = 0
directory = "images/final/"


# Get the names of all Pokemon that have "Fire" as their Type1 or Type2 attribute
water_pokemon_names = set(pokemon_data[(pokemon_data['Type1'] == 'Water') | (pokemon_data['Type2'] == 'Water')]['Name'])

# Filter train_ds to only include images of Pokemon that have "Fire" as either their Type1 or Type2 attribute

# iterate trough folder images\final\ and put in a numpy array image in folder that are in water_pokemon_names


# initialize an empty numpy array to store the images
for subdir, dirs, files in os.walk(directory):
    for name in dirs:
        if name in water_pokemon_names:
            num_images += len(os.listdir(os.path.join(subdir, name)))
images = np.empty((num_images, images_size[0], images_size[1], 4), dtype=np.float32)

i = 0
for subdir, dirs, files in os.walk(directory):
    # iterate through all subdirectories but only keep the ones that are in water_pokemon_names
    for name in dirs:
        if name in water_pokemon_names:
            # iterate through all images in the subdirectory
            for file in os.listdir(os.path.join(subdir, name)):
                # open the image and resize it
                img = Image.open(os.path.join(subdir, name, file))
                img = img.resize(images_size)
                # convert the image to RGBA
                img = img.convert('RGBA')
                # convert the image to a numpy array
                img = np.array(img)
                # add the image to the numpy array
                images[i] = img
                i += 1

water_labels = tf.ones((num_images,1))
water_ds = tf.data.Dataset.from_tensor_slices((images, water_labels))
water_ds = water_ds.batch(64)
water_ds = prepare(water_ds,shuffle=True, augment=True, GAN=True)


In [28]:
water_discriminator = define_discriminator()
water_generator = define_generator(100)
water_gan_model = define_gan(water_generator, water_discriminator)




In [29]:
with tf.device('/device:GPU:0'):
    train(water_generator, water_discriminator, water_gan_model, water_ds, 100, n_epochs=101)

NameError: name 'train' is not defined

In [2]:
# load model
model = tf.keras.models.load_model('generator_model_e1001.h5')



NameError: name 'tf' is not defined

In [1]:
# generate images
X = model.predict(generate_latent_points(100, 1))
plot_image(X[0], "image générée")



NameError: name 'model' is not defined